In [ ]:
! pip uninstall floquet
! cd .. # go to the repo root directory
! pip install -e .

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
import scqubits as scq
from cycler import cycler
from IPython.display import clear_output
import floquet as ft

num_states = 20

def get_H0_H1(qubit_instance: scq.GenericQubit) -> tuple[qt.Qobj, qt.Qobj]:
    hilbert_space = scq.HilbertSpace([qubit_instance])
    hilbert_space.generate_lookup()
    evals = hilbert_space["evals"][0][0:num_states]
    H0 = 2.0 * np.pi * qt.Qobj(np.diag(evals - evals[0]))
    H1 = hilbert_space.op_in_dressed_eigenbasis(qubit_instance.n_operator)
    return H0, H1

# Increase omega_d_values and chi_ac_values to see more dramatic speed difference

In [22]:
qubit_params = {"EJ": 22, "EC": 0.25, "ng": 0.25, "ncut": 41}
tmon = scq.Transmon(**qubit_params, truncated_dim=num_states)
state_indices = [0, 1]

H0, H1 = get_H0_H1(tmon)

omega_d_values = 2.0 * np.pi * np.linspace(3, 50, 200)
chi_ac_values = 2.0 * np.pi * np.linspace(0.0, 0.1, 200)
chi_to_amp = ft.ChiacToAmp(H0, H1, state_indices, omega_d_values)
drive_amplitudes = chi_to_amp.amplitudes_for_omega_d(chi_ac_values)

model = ft.Model(H0, H1, omega_d_values=omega_d_values, drive_amplitudes=drive_amplitudes)

# Using the new function

In [23]:
options = ft.Options(
    fit_range_fraction=0.5,  # split the fit into segments based on this fraction
    floquet_sampling_time_fraction=0.0,  # fraction of the period to compute the Floquet modes  # noqa E501
    fit_cutoff=4,  # polynomial cutoff
    overlap_cutoff=0.8,  # cutoff for excluding from the fit
    nsteps=30_000,  # qutip integration parameter
    num_cpus=20,  # number of cpus to use to compute Floquet modes in parallel
    save_floquet_modes=True,  # whether or not to save floquet modes themselves
    new_functions=True,
)


floquet_analysis = ft.FloquetAnalysis(
    model, state_indices=state_indices, options=options
)

data_vals_new = floquet_analysis.run()

calculating for amp_range_idx=0
_floquet_main_for_amp_range


Processing _run_floquet_and_calculate: 100%|██████████| 200/200 [00:35<00:00,  5.66it/s]


placing into
overlap_with_bare_states
displaced_states_fit ovlp_with_bare_states


Processing _fit_for_dim: 100%|██████████| 20/20 [00:09<00:00,  2.14it/s]


overlap_with_displaced_states new_coefficients


...

placing into
displaced_states_fit
displaced_states_fit intermediate_displaced_state_overlaps


Processing _fit_for_state_idx: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it]


overlap_with_displaced_states full_displaced_fit


Processing _run_overlap_displaced_batch: 100%|██████████| 80/80 [00:05<00:00, 15.78it/s]

finished in 2.216881998380025 minutes


# Old functions

In [24]:
from copy import deepcopy
options_old = deepcopy(options)
options_old.new_functions = False

floquet_analysis = ft.FloquetAnalysis(
    model, state_indices=state_indices, options=options_old
)

data_vals_old = floquet_analysis.run()

calculating for amp_range_idx=0
_floquet_main_for_amp_range


Processing _run_floquet_and_calculate:   0%|          | 0/200 [00:00<?, ?it/s]

Processing _run_floquet_and_calculate: 100%|██████████| 200/200 [00:35<00:00,  5.61it/s]


placing into
overlap_with_bare_states
displaced_states_fit ovlp_with_bare_states


Processing _fit_for_state_idx: 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


...

placing into
displaced_states_fit
displaced_states_fit intermediate_displaced_state_overlaps


Processing _fit_for_state_idx: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]


overlap_with_displaced_states full_displaced_fit


Processing _run_overlap_displaced:  17%|█▋        | 6997/40000 [03:37<17:05, 32.20it/s]


KeyboardInterrupt: 

# Check correctness

In [21]:
for key in data_vals_new.keys():
    assert np.allclose(data_vals_new[key], data_vals_old[key]), f'{key} is not close'